In [1]:
import cloudpickle as cpkl
from pathlib import Path

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import pymc_bart as pmb

from sklearn.model_selection import train_test_split
import dill

%config InlineBackend.figure_format = "retina"

In [5]:
# with open('test2.pkl', mode='rb') as file:
    # model = dill.load_session(file)
dill.load_session("test2.pkl")

AuthenticationError: digest sent was rejected

In [8]:
try:
    bikes = pd.read_csv(Path("..", "data", "bikes.csv"))
except FileNotFoundError:
    bikes = pd.read_csv(pm.get_data("bikes.csv"))

features = ["hour", "temperature", "humidity", "workingday"]

X = bikes[features]
Y = bikes["count"]


In [13]:
xt = X[0:10]
yt = Y[0:10]

In [24]:
with open("test_mu1.pkl", mode="rb") as file:
    mu = cpkl.load(file)

AuthenticationError: digest sent was rejected

In [9]:
with pm.Model() as model_bikes:
    xdata = pm.MutableData("xdata", X)
    a = pm.Exponential("a", 1)
    mu = pmb.BART("mu", xdata, np.log(Y), m=50)
    y = pm.NegativeBinomial("y", mu=pm.math.exp(mu), alpha=a, observed=Y, shape=xdata.shape[0])
    # idata_bikes = pm.sample(compute_convergence_checks=False, random_seed=99)
    # idata_bikes = pm.sample(random_seed=99, draws=100, tune=100, compute_convergence_checks=False)

In [12]:
idata = az.from_netcdf("test3.nc")
    

In [20]:
xt.shape

(10, 4)

In [22]:
with model_bikes:
    pm.set_data({"xdata":xt}, coords = {"obs":np.arange(0,xt.shape[0])})
    post = pm.sample_posterior_predictive(idata, var_names=["mu","y"])
post

Sampling: [mu, y]


ValueError: size does not match the broadcast shape of the parameters. (10,), (10,), (348,)
Apply node that caused the error: nbinom_rv{0, (0, 0), int64, True}(RandomGeneratorSharedVariable(<Generator(PCG64) at 0x7F1C413057E0>), MakeVector{dtype='int64'}.0, 4, a, Composite{(i1 / (exp(i0) + i1))}.0)
Toposort index: 5
Inputs types: [RandomGeneratorType, TensorType(int64, shape=(1,)), TensorType(int64, shape=()), TensorType(float64, shape=()), TensorType(float64, shape=(None,))]
Inputs shapes: ['No shapes', (1,), (), (), (348,)]
Inputs strides: ['No strides', (8,), (), (), (8,)]
Inputs values: [Generator(PCG64) at 0x7F1C413057E0, array([10]), array(4), array(2.10533087), 'not shown']
Outputs clients: [['output'], ['output']]

HINT: Re-running with most PyTensor optimizations disabled could provide a back-trace showing when this node was created. This can be done by setting the PyTensor flag 'optimizer=fast_compile'. If that does not work, PyTensor optimizations can be disabled with 'optim

AttributeError: 'InferenceData' object has no attribute 'predict'

In [10]:
# with open('test1.pkl', mode='rb') as file:
#    model4 = cpkl.load(file)

with open('test1.pkl', mode='rb') as file:
   model4 = cpkl.load(file)

AuthenticationError: digest sent was rejected